# 8. Compute Results Overview Tables - Reference

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

After filtering, the sequences are mapped with the reference replicons and the metrics are computed through custom python code. The metrics are calculated, filtered for a minimum length, for each replicon in the file provided input references. 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **COMPASS**
    - **(Breadth of) Coverage:** Ratio of covered sequence on the reference by aligned contigs;
    - **Multiplicity:** Ratio of the length of alignable assembled sequence to covered sequence on the reference;
    - **Validity:** Ratio of the length of the alignable assembled sequence to total basepairs in the aligned contigs;
    - **Parsimony:** Cost of the assembly (multiplicity over validity);
- **Contiguity**
    - **Contiguity:** longest single alignment between the assembly and the reference, relative to the reference length;
    - **NAx (where 0  < x  ⩽ 100):** Length for which the collection of aligned assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the reference replicon;
    - **NGx (where 0  < x  ⩽ 100):** Length for which the collection of aligned contigs of that length or longer covers at least a given percentage of the sequence of the reference.
    - **Lx (where 0  < x  ⩽ 100):** Minimal number of contigs that cover x % of the sequence of the reference;
- **Identity**
    - **Identity:** Ratio of identical basepairs in all aligned contigs to the reference;
    - **Lowest identity:** Identity of the lowest scoring contig to the reference.
    - **PLS**: Phred-like score per contig, per assembler.
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Imports

In [2]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np

## Global variables

In [23]:
ASSEMBLER_PROCESS_LIST = ["ABYSS", "BCALM2", "GATBMINIAPIPELINE", "METAHIPMER2", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"ABYSS": "AbYSS",
                   "BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "METAHIPMER2": "MetaHipMer2",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}

log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EMS', 'LNN', 'ERR2984773']

genomic_assemblers = ['ABySS', 'BCALM2', 'MINIA', 'SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer']
metagenomic_assemblers = ['MetaHipMer2','GATBMiniaPipeline', 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']
single_kmer = ['BCALM2', 'MINIA', 'ABySS']
multiple_kmer = ['SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer', 'GATBMiniaPipeline', 
                 'IDBA-UD', 'MEGAHIT', 'metaSPAdes', 'MetaHipMer2']

skipped_assemblers = ['ABySS', 'BCALM2', 'MINIA', 'VelvetOptimiser', 'MetaHipMer2']

REFERENCE_TO_NAME = {"Bacillus_subtilis":"<i>Bacillus subtilis</i>",
                    "Enterococcus_faecalis":"<i>Enterococcus faecalis</i>",
                    "Escherichia_coli":"<i>Escherichia coli</i>",
                    "Escherichia_coli_plasmid": np.nan,
                    "Lactobacillus_fermentum": "<i>Lactobacillus fermentum</i>",
                    "Listeria_monocytogenes": "<i>Listeria monocytogenes</i>",
                    "Pseudomonas_aeruginosa": "<i>Pseudomonas aeruginosa</i>",
                    "Salmonella_enterica": "<i>Salmonella enterica</i>",
                    "Staphylococcus_aureus": "<i>Staphylococcus aureus</i>",
                    "Staphylococcus_aureus_plasmid1": np.nan,
                    "Staphylococcus_aureus_plasmid2": np.nan,
                    "Staphylococcus_aureus_plasmid3": np.nan}

best_min = ['Ns','misassembled contigs','misassembly events']
best_min_exept_0 = ['contigs', 'L90'] 
best_max = ['LSA', 'NA50','NG50','breadth_of_coverage','identity','lowest_identity']

target_compass = {'multiplicity': 1,'parsimony':1, 'validity':1}
    
target_dict = {
    'Bacillus_subtilis': 4045677,
    'Enterococcus_faecalis': 2845392,
    'Escherichia_coli': 4765434,
    'Escherichia_coli_plasmid': 110007,
    'Lactobacillus_fermentum': 1905333,
    'Listeria_monocytogenes': 2992342,
    'Pseudomonas_aeruginosa': 6792330,
    'Salmonella_enterica': 4759746,
    'Staphylococcus_aureus': 2718780,
    'Staphylococcus_aureus_plasmid1': 6337,
    'Staphylococcus_aureus_plasmid2': 2216,
    'Staphylococcus_aureus_plasmid3': 2993
}

COLOURS = ['#5876c8', '#58AEC8', '#39B185', '#9CCB86', '#EEB479', '#E88471', '#a54765', '#a42a2a', '#835221']

## Load data

In [15]:
report_glob = glob.glob('../Results/*/*/report/pipeline_report_tables.json')
reference_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for reference, data in json_report[sample]['ReferenceTables'].items():
                for row in data:
                    for item in row:
                        reference_pipeline_metrics_df = reference_pipeline_metrics_df.append({'run': stats_run,
                                                                                        'sample': sample,
                                                                                        'assembler': item['assembler'],
                                                                                        'reference': reference,
                                                                                        'LSA': item['contiguity'],
                                                                                        'breadth_of_coverage': item['breadth_of_coverage'],
                                                                                        'multiplicity': item['multiplicity'],
                                                                                        'validity': item['validity'],
                                                                                        'parsimony': item['parsimony'],
                                                                                        'identity': item['identity'],
                                                                                        'lowest_identity': item['lowest_identity'],
                                                                                        'L90': item['L90'],
                                                                                        'contigs': item['aligned_contigs'],
                                                                                        'NA50': item['NA50'],
                                                                                        'NG50': item['NG50'],
                                                                                        'basepairs': item['aligned_basepairs'],
                                                                                        'Ns': item['Ns'],
                                                                                        'misassembled contigs': item['misassembled_contigs'],
                                                                                        'misassembly events': item['misassembly_events']},
                                                                                       ignore_index=True)

reference_pipeline_metrics_df['distribution'] = np.where(reference_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
reference_pipeline_metrics_df = reference_pipeline_metrics_df[reference_pipeline_metrics_df.distribution != 'Log']
reference_pipeline_metrics_df['type'] = np.where(reference_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
reference_pipeline_metrics_df['algorythm'] = np.where(reference_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events', 'multiplicity','validity','parsimony','identity','lowest_identity']] = reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events','multiplicity','validity','parsimony','identity','lowest_identity']].apply(pd.to_numeric)
display(reference_pipeline_metrics_df)

Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...
Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


,L90,LSA,NA50,NG50,Ns,assembler,basepairs,breadth_of_coverage,contigs,identity,...,misassembly events,multiplicity,parsimony,reference,run,sample,validity,distribution,type,algorythm
864,13.0,0.106260,295233.0,295233.0,0.0,ABySS,3989374.0,0.986083,22.0,0.999910,...,0.0,1.000305,1.000305,Bacillus_subtilis,run1,EMS,1.000000,Even,Genomic,Single k-mer De Bruijn graph
865,0.0,0.002419,2062.0,1589.0,0.0,BCALM2,2844464.0,0.703087,1467.0,1.000000,...,0.0,1.004273,1.004273,Bacillus_subtilis,run1,EMS,1.000000,Even,Genomic,Single k-mer De Bruijn graph
866,13.0,0.189977,263445.0,263445.0,0.0,GATBMiniaPipeline,3984103.0,0.984780,27.0,0.999864,...,0.0,1.000073,1.000094,Bacillus_subtilis,run1,EMS,0.999979,Even,Metagenomic,Multiple k-mer De Bruijn graph
867,18.0,0.084760,212969.0,212969.0,0.0,IDBA-UD,3977845.0,0.983233,34.0,0.997696,...,0.0,1.000061,1.000084,Bacillus_subtilis,run1,EMS,0.999977,Even,Metagenomic,Multiple k-mer De Bruijn graph
868,13.0,0.189990,263586.0,263586.0,0.0,MEGAHIT,3986471.0,0.985366,27.0,0.999421,...,3.0,0.999952,1.000269,Bacillus_subtilis,run1,EMS,0.999683,Even,Metagenomic,Multiple k-mer De Bruijn graph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,0.0,0.000000,0.0,0.0,0.0,MINIA,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,Staphylococcus_aureus_plasmid3,run2,EMS,0.000000,Even,Genomic,Single k-mer De Bruijn graph
2588,0.0,0.908119,2718.0,2718.0,0.0,SKESA,2718.0,0.908119,1.0,1.000000,...,0.0,1.000000,1.000000,Staphylococcus_aureus_plasmid3,run2,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph
2589,0.0,0.999666,2992.0,2992.0,0.0,SPAdes,2992.0,0.999666,1.0,1.000000,...,0.0,1.000000,1.000000,Staphylococcus_aureus_plasmid3,run2,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph
2590,0.0,0.964250,2886.0,2886.0,0.0,Unicycler,2886.0,0.964250,1.0,1.000000,...,0.0,1.000000,1.000000,Staphylococcus_aureus_plasmid3,run2,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph


In [16]:
reference_pipeline_metrics_df.columns

Index(['L90', 'LSA', 'NA50', 'NG50', 'Ns', 'assembler', 'basepairs',
       'breadth_of_coverage', 'contigs', 'identity', 'lowest_identity',
       'misassembled contigs', 'misassembly events', 'multiplicity',
       'parsimony', 'reference', 'run', 'sample', 'validity', 'distribution',
       'type', 'algorythm'],
      dtype='object')

In [17]:
reference_pipeline_metrics_df['sample'].unique()

array(['EMS', 'ERR2984773', 'ENN'], dtype=object)

In [18]:
reference_pipeline_metrics_df['assembler'].unique()

array(['ABySS', 'BCALM2', 'GATBMiniaPipeline', 'IDBA-UD', 'MEGAHIT',
       'MetaHipMer2', 'metaSPAdes', 'MINIA', 'SKESA', 'SPAdes',
       'Unicycler', 'VelvetOptimiser'], dtype=object)

In [19]:
reference_pipeline_metrics_df['reference'].unique()

array(['Bacillus_subtilis', 'Enterococcus_faecalis',
       'Escherichia_coli_plasmid', 'Escherichia_coli',
       'Lactobacillus_fermentum', 'Listeria_monocytogenes',
       'Pseudomonas_aeruginosa', 'Salmonella_enterica',
       'Staphylococcus_aureus', 'Staphylococcus_aureus_plasmid1',
       'Staphylococcus_aureus_plasmid2', 'Staphylococcus_aureus_plasmid3'],
      dtype=object)

## Save data


In [20]:
reference_pipeline_metrics_df.to_csv("Tables/Results/Reference metrics.csv")

### Reference stats per assembler - per sample

#### Values

In [24]:
best_stats_dfs = {}

for sample in reference_pipeline_metrics_df['sample'].unique():
    print(sample)
    best_stats_dfs[sample] = {}
    for reference in reference_pipeline_metrics_df['reference'].unique(): 
        print(reference)
        stats_per_assembler = pd.DataFrame()
        best_stats_per_assembler = pd.DataFrame()
        for assembler in reference_pipeline_metrics_df['assembler'].unique():

            row = {'Assembler': assembler, 
                 'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
                 'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}

            best_row = {'Assembler': assembler}

            describe_df = reference_pipeline_metrics_df[(reference_pipeline_metrics_df['reference'] == reference) & (reference_pipeline_metrics_df['assembler'] == assembler) & (reference_pipeline_metrics_df['sample'] == sample)].describe()
            for column in describe_df.columns:
                mean = describe_df.loc['mean',column]
                minimum = describe_df.loc['min',column]
                maximum = describe_df.loc['max',column]

                row[column] = "{} [{};{}]".format(round(mean, 2), round(minimum,2), round(maximum,2))

                if "basepairs" in column:
                    target = target_dict[reference]
                    best_row[column] = min(list(reference_pipeline_metrics_df[column][(reference_pipeline_metrics_df['reference'] == reference) & (reference_pipeline_metrics_df['assembler'] == assembler) & (reference_pipeline_metrics_df['sample'] == sample)]), key=lambda x:abs(x-target))
                
                elif column in target_compass.keys():
                    target=1
                    best_row[column] = min(list(reference_pipeline_metrics_df[column][(reference_pipeline_metrics_df['reference'] == reference) & (reference_pipeline_metrics_df['assembler'] == assembler) & (reference_pipeline_metrics_df['sample'] == sample)]), key=lambda x:abs(x-target))
                
                elif column in best_min_exept_0:
                    temp = [x for x in list(reference_pipeline_metrics_df[column][(reference_pipeline_metrics_df['reference'] == reference) & (reference_pipeline_metrics_df['assembler'] == assembler) & (reference_pipeline_metrics_df['sample'] == sample)]) if x != 0]
                    if len(temp) > 0:
                        best_row[column] = min(temp)
                    else:
                        best_row[column] = 0
                elif column in best_min:
                    best_row[column] = minimum
                elif column in best_max:
                    best_row[column] = maximum

            stats_per_assembler=stats_per_assembler.append(row, ignore_index=True)
            best_stats_per_assembler=best_stats_per_assembler.append(best_row, ignore_index=True)

        stats_per_assembler = stats_per_assembler.set_index(['Assembler','Algorithm', 'Type'])
        best_stats_per_assembler = best_stats_per_assembler.set_index('Assembler')
        display(stats_per_assembler)
        display(best_stats_per_assembler)
        stats_per_assembler.to_csv("Tables/Results/Reference metrics per assembler - {} {}.csv".format(sample, reference))
        best_stats_per_assembler.to_csv("Tables/Results/Best Reference metrics per assembler - {} {}.csv".format(sample, reference))
        best_stats_dfs[sample][reference] = best_stats_per_assembler

EMS
Bacillus_subtilis


,,,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,Algorithm,Type,,,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,13.0 [13.0;13.0],0.11 [0.11;0.11],295233.0 [295233.0;295233.0],295233.0 [295233.0;295233.0],0.0 [0.0;0.0],3989374.0 [3989374.0;3989374.0],0.99 [0.99;0.99],22.0 [22.0;22.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],0.0 [0.0;0.0],2062.0 [2062.0;2062.0],1589.0 [1589.0;1589.0],0.0 [0.0;0.0],2844464.0 [2844464.0;2844464.0],0.7 [0.7;0.7],1467.0 [1467.0;1467.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,13.0 [13.0;13.0],0.19 [0.19;0.19],263445.0 [263445.0;263445.0],263445.0 [263445.0;263445.0],0.0 [0.0;0.0],3984103.0 [3984103.0;3984103.0],0.98 [0.98;0.98],27.0 [27.0;27.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,18.0 [18.0;18.0],0.08 [0.08;0.08],212969.0 [212969.0;212969.0],212969.0 [212969.0;212969.0],0.0 [0.0;0.0],3977845.0 [3977845.0;3977845.0],0.98 [0.98;0.98],34.0 [34.0;34.0],1.0 [1.0;1.0],0.92 [0.92;0.92],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,13.0 [13.0;13.0],0.19 [0.19;0.19],263586.0 [263586.0;263586.0],263586.0 [263586.0;263586.0],0.0 [0.0;0.0],3986471.0 [3986471.0;3986471.0],0.99 [0.99;0.99],27.0 [27.0;27.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],3.0 [3.0;3.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,16.0 [16.0;16.0],0.1 [0.1;0.1],257424.0 [257424.0;257424.0],257424.0 [257424.0;257424.0],0.0 [0.0;0.0],3976387.0 [3976387.0;3976387.0],0.98 [0.98;0.98],28.0 [28.0;28.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,11.0 [11.0;11.0],0.19 [0.19;0.19],307964.0 [307964.0;307964.0],307964.0 [307964.0;307964.0],0.0 [0.0;0.0],3981174.0 [3981174.0;3981174.0],0.98 [0.98;0.98],23.0 [23.0;23.0],1.0 [1.0;1.0],0.97 [0.97;0.97],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MINIA,Single k-mer De Bruijn graph,Genomic,66.0 [66.0;66.0],0.03 [0.03;0.03],65138.0 [65138.0;65138.0],64960.0 [64960.0;64960.0],0.0 [0.0;0.0],3953200.0 [3953200.0;3953200.0],0.98 [0.98;0.98],117.0 [117.0;117.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
SKESA,Multiple k-mer De Bruijn graph,Genomic,14.0 [14.0;14.0],0.1 [0.1;0.1],269493.0 [269493.0;269493.0],263164.0 [263164.0;263164.0],0.0 [0.0;0.0],3971432.0 [3971432.0;3971432.0],0.98 [0.98;0.98],26.0 [26.0;26.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]


,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
ABySS,13.0,0.106260,295233.0,295233.0,0.0,3989374.0,0.986083,22.0,0.999910,0.998254,0.0,0.0,1.000305,1.000305,1.000000
BCALM2,0.0,0.002419,2062.0,1589.0,0.0,2844464.0,0.703087,1467.0,1.000000,1.000000,0.0,0.0,1.004273,1.004273,1.000000
GATBMiniaPipeline,13.0,0.189977,263445.0,263445.0,0.0,3984103.0,0.984780,27.0,0.999864,0.996848,0.0,0.0,1.000073,1.000094,0.999979
IDBA-UD,18.0,0.084760,212969.0,212969.0,0.0,3977845.0,0.983233,34.0,0.997696,0.922166,0.0,0.0,1.000061,1.000084,0.999977
MEGAHIT,13.0,0.189990,263586.0,263586.0,0.0,3986471.0,0.985366,27.0,0.999421,0.995636,1.0,3.0,0.999952,1.000269,0.999683
MetaHipMer2,16.0,0.097832,257424.0,257424.0,0.0,3976387.0,0.982873,28.0,1.000000,1.000000,0.0,0.0,1.000049,1.000049,1.000000
metaSPAdes,11.0,0.189944,307964.0,307964.0,0.0,3981174.0,0.984056,23.0,0.998251,0.971025,0.0,0.0,0.999981,0.999982,0.999998
MINIA,66.0,0.032648,65138.0,64960.0,0.0,3953200.0,0.977142,117.0,0.999993,0.999173,0.0,0.0,1.000284,1.000284,1.000000
SKESA,14.0,0.097783,269493.0,263164.0,0.0,3971432.0,0.981648,26.0,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000


Enterococcus_faecalis


,,,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,Algorithm,Type,,,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,11.0 [11.0;11.0],0.19 [0.19;0.19],240110.0 [240110.0;240110.0],240110.0 [240110.0;240110.0],0.0 [0.0;0.0],2821320.0 [2821320.0;2821320.0],0.99 [0.99;0.99],22.0 [22.0;22.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.02 [1.02;1.02],1.02 [1.02;1.02],1.0 [1.0;1.0]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],0.0 [0.0;0.0],1208.0 [1208.0;1208.0],0.0 [0.0;0.0],0.0 [0.0;0.0],224629.0 [224629.0;224629.0],0.08 [0.08;0.08],180.0 [180.0;180.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,13.0 [13.0;13.0],0.19 [0.19;0.19],235254.0 [235254.0;235254.0],235254.0 [235254.0;235254.0],0.0 [0.0;0.0],2807690.33 [2784031.0;2819520.0],0.99 [0.98;0.99],28.67 [28.0;29.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,28.0 [28.0;28.0],0.14 [0.14;0.14],113206.0 [113206.0;113206.0],113206.0 [113206.0;113206.0],0.0 [0.0;0.0],2811602.0 [2811602.0;2811602.0],0.99 [0.99;0.99],53.0 [53.0;53.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,13.0 [13.0;13.0],0.23 [0.23;0.23],235316.0 [235316.0;235316.0],238546.0 [238546.0;238546.0],0.0 [0.0;0.0],2810735.33 [2786540.0;2822833.0],0.99 [0.98;0.99],31.67 [31.0;32.0],1.0 [1.0;1.0],0.95 [0.95;0.95],1.0 [1.0;1.0],2.0 [2.0;2.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,27.0 [27.0;27.0],0.14 [0.14;0.14],118895.0 [118895.0;118895.0],114807.0 [114807.0;114807.0],0.0 [0.0;0.0],2787653.67 [2775796.0;2811369.0],0.98 [0.98;0.99],46.33 [46.0;47.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,11.0 [11.0;11.0],0.19 [0.19;0.19],238611.0 [238611.0;238611.0],238611.0 [238611.0;238611.0],0.0 [0.0;0.0],2786440.0 [2786440.0;2786440.0],0.98 [0.98;0.98],23.0 [23.0;23.0],1.0 [1.0;1.0],0.98 [0.98;0.98],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MINIA,Single k-mer De Bruijn graph,Genomic,104.33 [103.0;107.0],0.05 [0.05;0.05],30453.0 [30453.0;30453.0],30420.67 [30356.0;30453.0],0.0 [0.0;0.0],2758386.0 [2734580.0;2770289.0],0.97 [0.96;0.97],166.67 [166.0;167.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
SKESA,Multiple k-mer De Bruijn graph,Genomic,19.0 [19.0;19.0],0.14 [0.14;0.14],152709.0 [152709.0;152709.0],152709.0 [152709.0;152709.0],0.0 [0.0;0.0],2810284.0 [2810284.0;2810284.0],0.99 [0.99;0.99],40.0 [40.0;40.0],0.98 [0.98;0.98],0.37 [0.37;0.37],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.02 [1.02;1.02],0.98 [0.98;0.98]


...

,,,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,Algorithm,Type,,,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],1.0 [1.0;1.0],2215.0 [2215.0;2215.0],2215.0 [2215.0;2215.0],0.0 [0.0;0.0],2215.0 [2215.0;2215.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],1.0 [1.0;1.0],2215.0 [2215.0;2215.0],2215.0 [2215.0;2215.0],0.0 [0.0;0.0],2215.0 [2215.0;2215.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],1.0 [1.0;1.0],2215.0 [2215.0;2215.0],2215.0 [2215.0;2215.0],0.0 [0.0;0.0],2215.0 [2215.0;2215.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],1.0 [1.0;1.0],2213.0 [2213.0;2213.0],2213.0 [2213.0;2213.0],0.0 [0.0;0.0],2213.0 [2213.0;2213.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],0.94 [0.94;0.94],2076.0 [2076.0;2076.0],2076.0 [2076.0;2076.0],0.0 [0.0;0.0],2076.0 [2076.0;2076.0],0.94 [0.94;0.94],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],0.66 [0.66;0.66],1460.0 [1460.0;1460.0],1460.0 [1460.0;1460.0],0.0 [0.0;0.0],1460.0 [1460.0;1460.0],0.66 [0.66;0.66],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]


,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
ABySS,0.0,0.999549,2215.0,2215.0,0.0,2215.0,0.999549,1.0,1.000000,1.000000,0.0,0.0,1.0,1.000000,1.000000
BCALM2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
GATBMiniaPipeline,0.0,0.999549,2215.0,2215.0,0.0,2215.0,0.999549,1.0,1.000000,1.000000,0.0,0.0,1.0,1.000000,1.000000
IDBA-UD,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
MEGAHIT,0.0,0.999549,2215.0,2215.0,0.0,2215.0,0.999549,1.0,1.000000,1.000000,0.0,0.0,1.0,1.000000,1.000000
MetaHipMer2,0.0,0.998646,2213.0,2213.0,0.0,2213.0,0.998646,1.0,1.000000,1.000000,0.0,0.0,1.0,1.000000,1.000000
metaSPAdes,0.0,0.936823,2076.0,2076.0,0.0,2076.0,0.936823,1.0,0.999518,0.999518,0.0,0.0,1.0,1.000000,1.000000
MINIA,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
SKESA,0.0,0.658845,1460.0,1460.0,0.0,1460.0,0.658845,1.0,1.000000,1.000000,0.0,0.0,1.0,1.000000,1.000000


Staphylococcus_aureus_plasmid3


,,,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,Algorithm,Type,,,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],1.0 [1.0;1.0],2992.0 [2992.0;2992.0],2992.0 [2992.0;2992.0],0.0 [0.0;0.0],2992.0 [2992.0;2992.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],1.0 [1.0;1.0],2992.0 [2992.0;2992.0],2992.0 [2992.0;2992.0],0.0 [0.0;0.0],2992.0 [2992.0;2992.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],0.95 [0.95;0.95],2847.0 [2847.0;2847.0],2847.0 [2847.0;2847.0],0.0 [0.0;0.0],2847.0 [2847.0;2847.0],0.95 [0.95;0.95],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],1.0 [1.0;1.0],2992.0 [2992.0;2992.0],2992.0 [2992.0;2992.0],0.0 [0.0;0.0],2992.0 [2992.0;2992.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0.0;0.0],1.0 [1.0;1.0],2990.0 [2990.0;2990.0],2990.0 [2990.0;2990.0],0.0 [0.0;0.0],2990.0 [2990.0;2990.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,1.0 [1.0;1.0],0.56 [0.56;0.56],1677.0 [1677.0;1677.0],1677.0 [1677.0;1677.0],0.0 [0.0;0.0],2992.0 [2992.0;2992.0],1.0 [1.0;1.0],2.0 [2.0;2.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.02 [1.02;1.02],1.02 [1.02;1.02],1.0 [1.0;1.0]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0],0.0 [0.0;0.0]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0.0;0.0],0.7 [0.7;0.7],2110.0 [2110.0;2110.0],2110.0 [2110.0;2110.0],0.0 [0.0;0.0],2110.0 [2110.0;2110.0],0.7 [0.7;0.7],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]


,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
ABySS,0.0,0.999666,2992.0,2992.0,0.0,2992.0,0.999666,1.0,1.0,1.0,0.0,0.0,1.000000,1.000000,1.0
BCALM2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
GATBMiniaPipeline,0.0,0.999666,2992.0,2992.0,0.0,2992.0,0.999666,1.0,1.0,1.0,0.0,0.0,1.000000,1.000000,1.0
IDBA-UD,0.0,0.951220,2847.0,2847.0,0.0,2847.0,0.951220,1.0,1.0,1.0,0.0,0.0,1.000000,1.000000,1.0
MEGAHIT,0.0,0.999666,2992.0,2992.0,0.0,2992.0,0.999666,1.0,1.0,1.0,0.0,0.0,1.000000,1.000000,1.0
MetaHipMer2,0.0,0.998998,2990.0,2990.0,0.0,2990.0,0.998998,1.0,1.0,1.0,0.0,0.0,1.000000,1.000000,1.0
metaSPAdes,1.0,0.560307,1677.0,1677.0,0.0,2992.0,0.999666,2.0,1.0,1.0,0.0,0.0,1.018382,1.018382,1.0
MINIA,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
SKESA,0.0,0.704978,2110.0,2110.0,0.0,2110.0,0.704978,1.0,1.0,1.0,0.0,0.0,1.000000,1.000000,1.0


#### rank

In [36]:
rank_dfs = {}
for sample in best_stats_dfs:
    print(sample)
    rank_dfs[sample] = {}
    for reference in best_stats_dfs[sample]:
        if 'plasmid' not in reference:
            df = best_stats_dfs[sample][reference]

            rank_df = pd.DataFrame(index=df.index)

            for column in df.columns:
                rank_list = []

                if "basepairs" in column:
                    target = target_dict[reference]
                    for item in df[column]:
                        diff = 1 - abs(item-target) / target
                        rank_list.append(diff)
                    rank_df[column] = rank_list
                
                elif column in target_compass:
                    target = 1
                    for item in df[column]:
                        diff = 1- abs(item-target) / target
                        rank_list.append(diff)
                    rank_df[column] = rank_list
                
                elif column in best_min_exept_0:
                    for item in df[column]:
                        if item == 0:
                            diff=0
                        else:
                            diff = 1 - (item / df[column].max())
                        rank_list.append(diff)
                
                elif column in best_min:
                    for item in df[column]:
                        diff = 1 - (item / df[column].max())
                        rank_list.append(diff)
                else:
                    for item in df[column]:
                        diff = (item / df[column].max())
                        rank_list.append(diff)

                rank_df[column] = rank_list

            # in case assembly fails
            for assembler, row in df.iterrows():
                if row.sum() == 0:
                    rank_df.at[assembler, :]=0
            for to_skip in skipped_assemblers:
                rank_df = rank_df.drop(to_skip)

            rank_df = rank_df.fillna(1) #happens when all values are 0
            display(rank_df) 
            rank_df.to_csv("Tables/Results/Rank Global metrics per assembler - {}.csv".format(sample))
            rank_dfs[sample][reference]= rank_df


EMS


/tmp/ipykernel_3537/3537624744.py:38: RuntimeWarning:

invalid value encountered in double_scalars



,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.803030,0.999599,0.836094,0.836094,1.0,0.984780,0.998679,0.981595,0.999864,0.996848,1.0,1.0,0.999927,0.999906,0.999979
IDBA-UD,0.727273,0.445983,0.675899,0.675899,1.0,0.983233,0.997110,0.976823,0.997696,0.922166,1.0,1.0,0.999939,0.999916,0.999977
MEGAHIT,0.803030,0.999671,0.836542,0.836542,1.0,0.985366,0.999272,0.981595,0.999421,0.995636,0.0,0.0,0.999952,0.999731,0.999683
metaSPAdes,0.833333,0.999425,0.977384,0.977384,1.0,0.984056,0.997945,0.984322,0.998251,0.971025,1.0,1.0,0.999981,0.999982,0.999998
SKESA,0.787879,0.514503,0.855289,0.835203,1.0,0.981648,0.995503,0.982277,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000
SPAdes,0.833333,1.000000,1.000000,1.000000,1.0,0.984095,0.997984,0.985003,0.999995,0.999911,1.0,1.0,0.999922,0.999923,0.999999
Unicycler,0.833333,0.999289,0.999514,0.999514,1.0,0.982755,0.996625,0.986367,0.999981,0.999612,1.0,1.0,1.000000,1.000000,1.000000


/tmp/ipykernel_3537/3537624744.py:38: RuntimeWarning:

invalid value encountered in double_scalars



,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.877358,0.806869,0.623844,0.623844,1.0,0.990907,0.998826,0.844444,0.999990,0.999708,1.0,1.0,0.999617,0.999617,1.000000
IDBA-UD,0.735849,0.578086,0.300198,0.300198,1.0,0.988125,0.996021,0.705556,0.999974,0.998777,1.0,1.0,0.999772,0.999771,0.999999
MEGAHIT,0.877358,1.000000,0.624008,0.632574,1.0,0.992072,1.000000,0.827778,0.998077,0.950700,0.0,0.0,0.999977,0.999979,0.999956
metaSPAdes,0.896226,0.807076,0.632746,0.632746,1.0,0.979282,0.987108,0.872222,0.998336,0.980797,1.0,1.0,0.999838,0.999838,1.000000
SKESA,0.820755,0.577890,0.404952,0.404952,1.0,0.987661,0.995554,0.777778,0.984162,0.366461,0.0,0.5,1.000000,0.976641,0.977174
SPAdes,0.867925,0.806776,0.624926,0.570540,1.0,0.977576,0.985389,0.855556,0.999933,0.998727,1.0,1.0,0.999823,0.999821,0.999998
Unicycler,0.924528,0.999711,1.000000,1.000000,1.0,0.978188,0.986005,0.888889,0.999999,0.999990,1.0,1.0,0.999841,0.999841,1.000000


...

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.853333,0.865494,0.784073,0.784073,1.0,0.986244,0.996399,0.850394,0.999979,0.999574,1.0,1.0,0.999990,0.999990,1.000000
IDBA-UD,0.826667,0.801976,0.568570,0.568570,1.0,0.986637,0.996796,0.795276,0.999891,0.997226,1.0,1.0,0.999913,0.999913,1.000000
MEGAHIT,0.893333,0.865575,1.000000,1.000000,1.0,0.989584,0.999773,0.850394,0.999446,0.990650,1.0,1.0,0.999685,0.999697,0.999987
metaSPAdes,0.880000,0.865383,0.783975,0.783975,1.0,0.976904,0.986963,0.874016,0.999580,0.997236,1.0,1.0,0.999962,0.999962,1.000000
SKESA,0.840000,0.862988,0.783780,0.783780,1.0,0.972489,0.982503,0.811024,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000
SPAdes,0.866667,0.743367,0.670830,0.670830,1.0,0.974766,0.984802,0.874016,0.999580,0.997541,1.0,1.0,0.999753,0.999753,1.000000
Unicycler,0.893333,0.997148,0.783749,0.783749,1.0,0.974016,0.984045,0.874016,0.998116,0.969859,1.0,1.0,0.999991,0.999995,0.999986


,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.921922,0.314815,0.335892,0.262063,1.0,0.987016,0.996118,0.881513,0.999482,0.957340,0.625,0.600000,0.999158,0.998692,0.999534
IDBA-UD,0.928929,0.379058,0.330240,0.257652,1.0,0.985660,0.994750,0.885082,0.998614,0.861648,0.750,0.733333,0.999308,0.999171,0.999864
MEGAHIT,0.970971,0.522165,0.924849,0.721565,1.0,0.990862,1.000000,0.941470,0.999828,0.989028,1.000,1.000000,0.999591,0.999591,0.999999
metaSPAdes,0.975976,1.000000,0.975694,0.761235,1.0,0.990271,0.999403,0.960029,0.999508,0.981866,1.000,1.000000,0.999883,0.999883,1.000000
SKESA,0.329329,0.048096,0.038268,0.029216,1.0,0.963658,0.972545,0.334047,0.998822,0.250734,0.125,0.000000,0.999949,0.996893,0.996851
SPAdes,0.973974,0.947181,0.861241,0.671939,1.0,0.988865,0.997984,0.952177,0.999529,0.990299,0.750,0.800000,0.999973,0.999974,0.999999
Unicycler,0.974975,0.944280,0.970408,0.705754,1.0,0.987023,0.996125,0.945039,0.999991,0.999445,1.000,1.000000,1.000000,1.000000,1.000000


,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.438596,0.520606,0.266415,0.259010,1.0,0.975653,0.987917,0.835642,0.999949,0.991722,1.000000,1.000000,0.998487,0.998486,0.999999
IDBA-UD,0.000000,0.335826,0.133300,0.139433,1.0,0.956285,0.968305,0.634349,0.994826,0.655810,0.000000,0.000000,0.992080,0.984494,0.992530
MEGAHIT,0.725146,0.711129,0.591501,0.591501,1.0,0.981011,0.993343,0.904894,0.999158,0.929582,1.000000,1.000000,0.998757,0.998692,0.999935
metaSPAdes,0.853801,1.000000,1.000000,1.000000,1.0,0.982070,0.994415,0.948292,0.998099,0.934869,1.000000,1.000000,0.999874,0.999835,0.999960
SKESA,0.157895,0.279439,0.191062,0.189457,1.0,0.959395,0.971454,0.773777,0.996346,0.221785,0.714286,0.833333,0.998566,0.990178,0.991694
SPAdes,0.730994,0.999828,0.521350,0.518663,1.0,0.969381,0.981566,0.915974,0.999293,0.987168,1.000000,1.000000,0.999841,0.999839,0.999998
Unicycler,0.795322,0.692563,0.793837,0.779942,1.0,0.974371,0.986619,0.917821,0.999713,0.991144,1.000000,1.000000,1.000000,1.000000,1.000000


/tmp/ipykernel_3537/3537624744.py:38: RuntimeWarning:

invalid value encountered in double_scalars



,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.651786,0.175415,0.176898,0.147971,1.0,0.982306,0.991376,0.628571,1.000000,1.000000,1.0,1.0,0.999330,0.999330,1.000000
IDBA-UD,0.616071,0.175413,0.147539,0.147539,1.0,0.980518,0.989571,0.577143,0.999978,0.998542,1.0,1.0,0.999994,0.999994,1.000000
MEGAHIT,0.875000,0.350081,0.503760,0.590760,1.0,0.986211,0.995317,0.771429,0.999509,0.992685,0.0,0.0,0.999851,0.999623,0.999474
metaSPAdes,0.928571,1.000000,1.000000,1.000000,1.0,0.986268,0.995374,0.891429,0.999732,0.996403,1.0,1.0,0.999974,0.999974,1.000000
SKESA,0.616071,0.169394,0.146985,0.146985,1.0,0.975447,0.984453,0.582857,1.000000,0.999979,1.0,1.0,0.999982,0.999982,1.000000
SPAdes,0.875000,0.554028,0.594146,0.594146,1.0,0.982230,0.991299,0.862857,0.996927,0.934990,1.0,1.0,0.999987,0.999986,0.999973
Unicycler,0.875000,0.810958,0.618615,0.454454,1.0,0.982140,0.991208,0.857143,0.999991,0.999788,1.0,1.0,0.999966,0.999966,1.000000


### radar plots

In [37]:
for sample in rank_dfs:
    print(sample)
    fig = make_subplots(rows=2, cols=4, specs=[[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}],[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]], subplot_titles= ['<i>' + x.replace('_',' ') + '</i><br>' for x in sorted(rank_dfs[sample])])
    col_n=1
    row_n=1
    for reference in sorted(rank_dfs[sample]):
        print(reference)
        print(col_n,row_n)
        showlegend=True if col_n==1 and row_n==1 else False

        df = rank_dfs[sample][reference]
        i= 0

        for assembler, row in df.iterrows():

            row_data = []

            for col in df.columns:
                row_data.append(row.loc[:].at[col])

            fig.add_trace(go.Scatterpolar(r=row_data,
                                          theta=list([z.replace('_',' ') for z in df.columns]), mode='lines+markers',
                                          marker=dict(color=COLOURS[i], size=12), 
                                          marker_line_color="black", 
                                          marker_line_width=2,
                                          opacity=0.6,
                                          name=assembler, line=dict(color=COLOURS[i]), showlegend=showlegend),
                          col=col_n, row=row_n)

            i+=1
        if row_n == 2:
            row_n = 1
            col_n += 1
        else:
            row_n += 1
    
    fig.update_layout(polar=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar2=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar3=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar4=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar5=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar6=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar7=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar8=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5))
    
    
    fig.update_layout(title=dict(text='<b>'+sample,
                                 font=dict(size=42, color='black'),
                                 y=0.97,
                                 x=0.5,
                                 xanchor='center',
                                 yanchor='top'),
                     font=dict(size=14))
    fig.layout.annotations[0].update(y=0.54, font=dict(size=20,color="black"))
    fig.layout.annotations[1].update(y=0.54, font=dict(size=20,color="black"))
    fig.layout.annotations[2].update(y=0.54, font=dict(size=20,color="black"))
    fig.layout.annotations[3].update(y=0.54, font=dict(size=20,color="black"))
    fig.layout.annotations[4].update(y=0.42, font=dict(size=20,color="black"))
    fig.layout.annotations[5].update(y=0.42, font=dict(size=20,color="black"))
    fig.layout.annotations[6].update(y=0.42, font=dict(size=20,color="black"))
    fig.layout.annotations[7].update(y=0.42, font=dict(size=20,color="black"))

    fig.show()
    plot(fig, filename='Plots/Reference Metrics/Radial plot - {}.html'.format(sample), auto_open=False)



EMS
Bacillus_subtilis
1 1
Enterococcus_faecalis
1 2
Escherichia_coli
2 1
Lactobacillus_fermentum
2 2
Listeria_monocytogenes
3 1
Pseudomonas_aeruginosa
3 2
Salmonella_enterica
4 1
Staphylococcus_aureus
4 2


ERR2984773
Bacillus_subtilis
1 1
Enterococcus_faecalis
1 2
Escherichia_coli
2 1
Lactobacillus_fermentum
2 2
Listeria_monocytogenes
3 1
Pseudomonas_aeruginosa
3 2
Salmonella_enterica
4 1
Staphylococcus_aureus
4 2


ENN
Bacillus_subtilis
1 1
Enterococcus_faecalis
1 2
Escherichia_coli
2 1
Lactobacillus_fermentum
2 2
Listeria_monocytogenes
3 1
Pseudomonas_aeruginosa
3 2
Salmonella_enterica
4 1
Staphylococcus_aureus
4 2


In [32]:
for sample in rank_dfs:
    for reference in rank_dfs[sample]:
        fig = go.Figure()
        df = rank_dfs[sample][reference]
        for assembler in df.index:

            fig.add_trace(go.Box(x=df.loc[[assembler]].values[0],
                                     name=assembler, boxpoints='all',jitter=1, pointpos=0,
                                     boxmean=False, fillcolor='#D3D3D3', line=dict(color='#000000'),
                                    marker=dict(color='rgba(178,37,34,0.7)', size=6)))
        fig.update_layout(showlegend=False, xaxis_title="Score",
                              plot_bgcolor='rgb(255,255,255)', xaxis=dict(zeroline=False, gridcolor='#DCDCDC'),
                         title='{} - {}'.format(sample, reference))
        fig.show()
